<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/socio-economic/HousingValuesGeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pygeohash

  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6162 sha256=19315b6cdcfe750de3df48b4e0388bc47f09e41f7e54407ad78c4a6de436d0c0
  Stored in directory: /root/.cache/pip/wheels/3f/5f/14/989d83a271207dda28232746d63e737a2dbd88ea7f7a9db807
Successfully built pygeohash


In [6]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pygeohash as pgh
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/'

In [7]:
SAM_addresses = pd.read_csv(DATA_ROOT + 'Live_Street_Address_Management__SAM__Addresses.csv')
assessment = pd.read_csv(DATA_ROOT + '2020-property-assessment.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,9,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,15,27,28,29,34,35,36,37,38,39,40,41,42,44,45,46,47,48,58,59,64,65,66,67,68,69,70,72,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
SAM_addresses.shape

(399248, 27)

In [9]:
assessment.shape

(175052, 75)

In [10]:
SAM_addresses[SAM_addresses['PARCEL']==2200078000]

,X,Y,SAM_ADDRESS_ID,RELATIONSHIP_TYPE,BUILDING_ID,FULL_ADDRESS,STREET_NUMBER,IS_RANGE,RANGE_FROM,RANGE_TO,UNIT,FULL_STREET_NAME,STREET_ID,STREET_PREFIX,STREET_BODY,STREET_SUFFIX_ABBR,STREET_FULL_SUFFIX,STREET_SUFFIX_DIR,STREET_NUMBER_SORT,MAILING_NEIGHBORHOOD,ZIP_CODE,X_COORD,Y_COORD,SAM_STREET_ID,WARD,PRECINCT_WARD,PARCEL
132036,-71.124341,42.360299,150718,1,143177,30 Windom St,30,0,NaN,NaN,NaN,Windom St,4535,,Windom,St,Street,,30.0,Allston,2134.0,757696.825869,2.956542e+06,4535,22,2201,2.200078e+09


In [11]:
assessment[assessment['PID']==2200078000]

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,OWN_OCC,OWNER,MAIL_ADDRESSEE,MAIL_ADDRESS,MAIL CS,MAIL_ZIPCODE,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,LAND_SF,YR_BUILT,YR_REMOD,GROSS_AREA,LIVING_AREA,NUM_FLOORS,STRUCTURE_CLASS,R_BLDG_STYL,R_ROOF_TYP,R_EXT_FIN,R_TOTAL_RMS,R_BDRMS,R_FULL_BTH,R_HALF_BTH,R_BTH_STYLE,R_BTH_STYLE2,R_BTH_STYLE3,R_KITCH,R_KITCH_STYLE,R_KITCH_STYLE2,R_KITCH_STYLE3,R_HEAT_TYP,R_AC,R_FPLACE,R_EXT_CND,R_OVRALL_CND,R_INT_CND,R_INT_FIN,R_VIEW,S_NUM_BLDG,S_BLDG_STYL,S_UNIT_RES,S_UNIT_COM,S_UNIT_RC,S_EXT_FIN,S_EXT_CND,U_BASE_FLOOR,U_NUM_PARK,U_CORNER,U_ORIENT,U_TOT_RMS,U_BDRMS,U_FULL_BTH,U_HALF_BTH,U_BTH_STYLE,U_BTH_STYLE2,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW
163062,2200078000,NaN,2200078000,30,WINDOM,ST,NaN,2134.0,101,R1,Y,ANDERSON PRISCILLA R,C/O PRISCILLA ANDERSON,30 WINDOM,ALLSTON MA,2134,189400,350700,540100,5703.45,3400,1900.0,2013.0,1940.0,1128.0,2.0,NaN,CL - Colonial,L - Gambrel,M - Vinyl,6.0,3.0,1.0,1.0,M - Modern,M - Modern,NaN,1F - 1 Full Eat In K,M - Modern,NaN,NaN,F - Forced Hot Air,N - None,0.0,A - Average,A - Average,G - Good,N - Normal,A - Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
assessment_SAM = pd.merge(assessment, SAM_addresses, left_on='PID',right_on="PARCEL")

In [13]:
assessment_SAM.shape

(406162, 102)

In [14]:
assessment_SAM_reduced = assessment_SAM[['PID', 'AV_LAND', 'AV_BLDG', 'AV_TOTAL', 'X', 'Y']]

In [15]:
assessment_SAM_reduced.shape

(406162, 6)

In [16]:
assessment_SAM_reduced["GEOHASH"] = [pgh.encode(row["Y"],row["X"]) for idx, row in assessment_SAM_reduced.iterrows()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
assessment_SAM_reduced.head()

,PID,AV_LAND,AV_BLDG,AV_TOTAL,X,Y,GEOHASH
0,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
1,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
2,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
3,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
4,100274000,0,745300,745300,-71.027641,42.380959,drt3pepqbp29


In [18]:
pgh.geohash_haversine_distance(assessment_SAM_reduced.iloc[0,6], assessment_SAM_reduced.iloc[4, 6])

28.940991081356703

In [19]:
from geopy import distance

In [20]:
place1 = (assessment_SAM_reduced.iloc[0,5], assessment_SAM_reduced.iloc[0,4])
place2 = (assessment_SAM_reduced.iloc[4,5], assessment_SAM_reduced.iloc[4,4])
print(distance.distance(place1, place2).m)

29.016173010195022


In [21]:
origin = assessment_SAM_reduced[assessment_SAM_reduced['PID']==2200078000]['GEOHASH'].values[0]
origin

'drt2vx2ydspb'

In [22]:
assessment_SAM_reduced[(test_dist<20) & (assessment_SAM_reduced['AV_TOTAL']>0)].mean()['AV_TOTAL']

NameError: ignored

In [23]:
boston_crime_2020 = pd.read_csv(DATA_ROOT + 'boston_crime_2020.csv')
boston_crime_2020.shape

(50066, 30)

In [25]:
# boston_crime_2020.drop(columns=['Unnamed: 0'], inplace=True)
boston_crime_2020

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,GEOHASH,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME
0,S97333701,3301,VERBAL DISPUTE,C6,915,0.0,2020-07-18 14:34:00,2020,7,Saturday,14,NaN,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)",773.150,16450,0.047,0,Other,drt2z4jfz5kp,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550
1,202080441,3301,VERBAL DISPUTE,C6,231,0.0,2020-10-31 02:41:00,2020,10,Saturday,2,NaN,E SEVENTH ST,NaN,NaN,"(0.00000000, 0.00000000)",773.150,16450,0.047,0,Other,000000000000,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550
2,202080246,3301,VERBAL DISPUTE,C6,914,0.0,2020-10-30 10:51:00,2020,10,Friday,10,NaN,GENERAL LAWRENCE J LOGAN WAY,NaN,NaN,"(0.00000000, 0.00000000)",773.150,16450,0.047,0,Other,000000000000,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550
3,202080065,3301,VERBAL DISPUTE,C6,186,0.0,2020-10-29 16:30:00,2020,10,Thursday,16,NaN,BOSTON ST,42.327827,-71.058000,"(42.32782700, -71.05800000)",773.150,16450,0.047,0,Other,drt2z191kwfg,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550
4,202079731,3301,VERBAL DISPUTE,C6,936,0.0,2020-10-28 13:52:00,2020,10,Wednesday,13,NaN,ORTON-MAROTTA WAY,42.337318,-71.053832,"(42.33731800, -71.05383200)",773.150,16450,0.047,0,Other,drt2z5k0v8ng,0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50061,202003101,1415,GRAFFITI,D4,165,0.0,2020-01-12 09:30:00,2020,1,Sunday,9,NaN,HARRISON AVE,42.343297,-71.064581,"(42.34329700, -71.06458100)",1971.395,70570,0.098,0,Other,drt2yuk7e4xv,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975
50062,202036432,2642,TRUANCY / RUNAWAY,D4,273,0.0,2020-05-27 02:00:00,2020,5,Wednesday,2,NaN,CAMDEN ST,42.337441,-71.080576,"(42.33744100, -71.08057600)",1971.395,70570,0.098,0,Other,drt2ye29qx10,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975
50063,202017085,2642,TRUANCY / RUNAWAY,D4,271,0.0,2020-03-02 06:27:00,2020,3,Monday,6,NaN,HARRISON AVE,42.334018,-71.076381,"(42.33401800, -71.07638100)",1971.395,70570,0.098,0,Other,drt2ydev82cj,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975
50064,202056504,1603,PROSTITUTION - ASSISTING OR PROMOTING,D4,,0.0,2020-08-07 19:24:00,2020,8,Friday,19,NaN,MASSACHUSETTS AVE,42.332855,-71.072624,"(42.33285500, -71.07262400)",1971.395,70570,0.098,0,Prostitution,drt2ydqqzb4k,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975


In [26]:
boston_crime_2020_reduced = boston_crime_2020.dropna(axis=0, subset=['LATITUDE', 'LONGITUDE'])
boston_crime_2020_reduced.shape

(48750, 30)

In [42]:
# define function to apply across the GEOHASH of 
from collections import defaultdict

# setting up home value dict (to 118 meter precision) with geohash
home_value_dict = defaultdict(list)
for idx, row in assessment_SAM_reduced.iterrows():
    home_value_dict[row['GEOHASH'][:7]].append(row['AV_TOTAL'])




In [49]:
# function for getting values from the dictionary from geohash
from statistics import mean

def get_prop_values(geohash, home_values):
  try:
    mean_property_value = mean(home_values[geohash[:7]])
  except:
    mean_property_value = 0

  return mean_property_value

In [50]:
# prop_values_test = get_prop_values(origin, home_value_dict)
# this should create list of the property values near 
LOCAL_PROP_VALUES = [get_prop_values(row["GEOHASH"], home_value_dict) for idx, row in boston_crime_2020_reduced.iterrows()]

In [54]:
boston_crime_2020_reduced['LOCAL_PROP_VALUES'] = LOCAL_PROP_VALUES

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
boston_crime_2020_reduced.tail(10)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LATITUDE,LONGITUDE,LOCATION,COVID_POSITIVE_COUNT,COVID_TESTED_COUNT,COVID_POSITIVE_RATE,VIOLENT_CRIME,OFFENSE_TYPE,GEOHASH,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME,LOCAL_PROP_VALUES
50056,202012584,1415,GRAFFITI,D4,157,0.0,2020-02-15 08:30:00,2020,2,Saturday,8,NaN,SHAWMUT AVE,42.339537,-71.074724,"(42.33953700, -71.07472400)",1971.395,70570,0.098,0,Other,drt2yesvwvu8,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,1.590019e+06
50057,202012578,1415,GRAFFITI,D4,167,0.0,2020-02-14 17:00:00,2020,2,Friday,17,NaN,HARRISON AVE,42.339541,-71.069408,"(42.33954100, -71.06940800)",1971.395,70570,0.098,0,Other,drt2yg8vdy2r,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,5.482419e+07
50058,202005522,1415,GRAFFITI,D4,167,0.0,2020-01-21 14:30:00,2020,1,Tuesday,14,NaN,HARRISON AVE,42.339541,-71.069408,"(42.33954100, -71.06940800)",1971.395,70570,0.098,0,Other,drt2yg8vdy2r,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,5.482419e+07
50059,202005164,1415,GRAFFITI,D4,620,0.0,2020-01-20 05:20:00,2020,1,Monday,5,NaN,MASSACHUSETTS AVE,42.344395,-71.086320,"(42.34439500, -71.08632000)",1971.395,70570,0.098,0,Other,drt2yks9bxmg,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,2.162086e+08
50060,202004532,1415,GRAFFITI,D4,,0.0,2020-01-17 10:00:00,2020,1,Friday,10,NaN,TREMONT ST,42.345902,-71.069463,"(42.34590200, -71.06946300)",1971.395,70570,0.098,0,Other,drt2yubfc2r2,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,6.169746e+05
50061,202003101,1415,GRAFFITI,D4,165,0.0,2020-01-12 09:30:00,2020,1,Sunday,9,NaN,HARRISON AVE,42.343297,-71.064581,"(42.34329700, -71.06458100)",1971.395,70570,0.098,0,Other,drt2yuk7e4xv,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,4.155917e+06
50062,202036432,2642,TRUANCY / RUNAWAY,D4,273,0.0,2020-05-27 02:00:00,2020,5,Wednesday,2,NaN,CAMDEN ST,42.337441,-71.080576,"(42.33744100, -71.08057600)",1971.395,70570,0.098,0,Other,drt2ye29qx10,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,4.489371e+06
50063,202017085,2642,TRUANCY / RUNAWAY,D4,271,0.0,2020-03-02 06:27:00,2020,3,Monday,6,NaN,HARRISON AVE,42.334018,-71.076381,"(42.33401800, -71.07638100)",1971.395,70570,0.098,0,Other,drt2ydev82cj,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,1.196902e+06
50064,202056504,1603,PROSTITUTION - ASSISTING OR PROMOTING,D4,,0.0,2020-08-07 19:24:00,2020,8,Friday,19,NaN,MASSACHUSETTS AVE,42.332855,-71.072624,"(42.33285500, -71.07262400)",1971.395,70570,0.098,0,Prostitution,drt2ydqqzb4k,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,5.152512e+07
50065,202009470,2622,KIDNAPPING/CUSTODIAL KIDNAPPING/ ABDUCTION,D4,273,0.0,2020-02-03 11:00:00,2020,2,Monday,11,NaN,TREMONT ST,42.338654,-71.081456,"(42.33865400, -71.08145600)",1971.395,70570,0.098,1,Kidnapping,drt2ye80d10u,0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,8.074521e+05
